<a href="https://colab.research.google.com/github/rc-dbe/bigdatacertification/blob/master/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


*Hands-on of Big Data Analyst with TuV Certified Qualification*


---




# Optimization

<div>
<p>A manufacturer is producing sausages!</p>
<p>Here are the following ingredients available:</p>
<table>
<thead>
<tr>
<th>Ingredient</th>
<th>Cost ($/kg)</th>
<th>Availability (kg)</th>
</tr>
</thead>
<tbody>
<tr>
<td>Beef</td>
<td>5.72</td>
<td>30</td>
</tr>
<tr>
<td>Wheat</td>
<td>2.16</td>
<td>20</td>
</tr>
<tr>
<td>Starch</td>
<td>1.5</td>
<td>17</td>
</tr>
</tbody>
</table>
<p>There are 2 types of sausage:</p>
<ul>
<li>Economy (&gt;40% Beef)</li>
<li>Premium (&gt;60% Beef)</li>
</ul>
<p>Every piece of sausage is 50 grams (0.05 kg).</p>
<p>According to product specification, starch should be 25% of the total components per item.</p>
<p>A butcher has been contracted, 23 kg beef have already been purchased.</p>
<p>Demand: 350 economy sausages and 500 premium sausages.</p>
<p>What is the most cost effective blend for production.</p>
</div>
</div>
</div>

<div>
<div> 
<p>The model for the problem</p>
<p>$
  b_e = \text{Beef in the economy sausages (kg)} \\
  w_e = \text{Wheat in the economy sausages (kg)} \\
  s_e = \text{Starch in the economy sausages (kg)} \\
  b_p = \text{Beef in the premium sausages (kg)}  \\
  w_p = \text{Wheat in the premium sausages (kg)}  \\
  s_p = \text{Starch in the premium sausages (kg)}  \\
$</p>
<p>We want to minimise costs such that:</p>
<p>$\text{Cost} = 5.72(b_e + b_p) + 2.16(w_e + w_p) + 1.5(s_e + s_p)$</p>
<p>With the following constraints:</p>
<p>$
 b_e + w_e + s_e = 350 \times 0.05 \\
 b_p + w_p + s_p = 500 \times 0.05 \\
 b_e \geq 0.4(p_e + w_e + s_e) \\ 
 b_p \geq 0.6(p_p + w_p + s_p) \\
 s_e \leq 0.25(p_e + w_e + s_e) \\ 
 s_p \leq 0.25(p_p + w_p + s_p) \\ 
 b_e + b_p \leq 30 \\ 
 w_e + w_p \leq 20 \\
 s_e + s_p \leq 17 \\
 b_e + b_p \geq 23 \\
 $</p>
</div>
</div>
</div>

In [0]:
# Instal pulp package
!pip install pulp
import pulp

In [0]:
# Instantiate our problem class
model = pulp.LpProblem("Cost minimising blending problem", pulp.LpMinimize)

Create a couple of lists from which to create tuple indices, so we don't have to assign each of all 6 decision variables separately.

In [0]:
# Construct our decision variable lists
sausage_types = ['economy', 'premium']
ingredients = ['beef', 'wheat', 'starch']

Each of decision variables have similar characteristics (lower bound of 0, continuous variables). <br />
Therefore we can use PuLP’s LpVariable object’s dict functionality, we can provide our tuple indices.
These tuples will be keys for the ing_weight dict of decision variables

In [0]:
ing_weight = pulp.LpVariable.dicts("weight kg",
                                     ((i, j) for i in sausage_types for j in ingredients),
                                     lowBound=0,
                                     cat='Continuous')

PuLP provides an lpSum vector calculation for the sum of a list of linear expressions.

In [0]:
# Objective Function
model += (
    pulp.lpSum([
        5.72 * ing_weight[(i, 'beef')]
        + 2.16 * ing_weight[(i, 'wheat')]
        + 1.5 * ing_weight[(i, 'starch')]
        for i in sausage_types])
)

The list comprehensions allows for scaling up to many ingredients or sausage types

In [0]:
# Constraints
# 350 economy and 500 premium sausages at 0.05 kg
model += pulp.lpSum([ing_weight['economy', j] for j in ingredients]) == 350 * 0.05
model += pulp.lpSum([ing_weight['premium', j] for j in ingredients]) == 500 * 0.05

# Economy has >= 40% beef, premium has >= 60% beef
model += ing_weight['economy', 'beef'] >= (
    0.4 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

model += ing_weight['premium', 'beef'] >= (
    0.6 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# Sausages must be <= 25% starch
model += ing_weight['economy', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

model += ing_weight['premium', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# We have at most 30 kg of beef, 20 kg of wheat and 17 kg of starch available
model += pulp.lpSum([ing_weight[i, 'beef'] for i in sausage_types]) <= 30
model += pulp.lpSum([ing_weight[i, 'wheat'] for i in sausage_types]) <= 20
model += pulp.lpSum([ing_weight[i, 'starch'] for i in sausage_types]) <= 17

# We have at least 23 kg of beef to use up
model += pulp.lpSum([ing_weight[i, 'beef'] for i in sausage_types]) >= 23

In [0]:
# Solve our problem
model.solve()
pulp.LpStatus[model.status]

In [0]:
for var in ing_weight:
    var_value = ing_weight[var].varValue
    print ("The weight of {0} in {1} sausages is {2} kg".format(var[1], var[0], var_value))

In [0]:
total_cost = pulp.value(model.objective)

print ("The total cost is ${} for 350 economy sausages and 500 premium sausages".format(round(total_cost, 2)))